<a href="https://colab.research.google.com/github/ArtemAvanesov/Face-recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import urllib.request
from progressbar import *
import socket

# Чтение данных 

[Ссылка на датасет FaceScrub](http://vintage.winklerbros.net/facescrub.html "FaceScrub")   

<img align="left" width="580" height="200" src="http://vintage.winklerbros.net/Images/facescrub.jpg">   

In [0]:
base_dir = "/content/drive/My Drive/Colab/Roonyx/Face recognition"
folder_for_images = base_dir + "/faceScrub/images"
!mkdir '$folder_for_images'

f = open(base_dir + "/faceScrub/facescrub_actors.txt", 'r')
actors = []
for line in f:
    actor_info = line.split('\t')
    actors.append([actor_info[0], actor_info[3]])
actors = np.array(actors)
f.close()

f = open(base_dir + "/faceScrub/facescrub_actresses.txt", 'r')
actresses = []
for line in f:
    actresses_info = line.split('\t')
    actresses.append([actresses_info[0], actresses_info[3]])
actresses = np.array(actresses)
f.close()

# Пример данных

In [22]:
pd.DataFrame(data=actors[1:11], columns=actors[0])

,name,url
0,Aaron Eckhart,http://upload.wikimedia.org/wikipedia/commons/...
1,Aaron Eckhart,http://movies.dosthana.com/sites/default/files...
2,Aaron Eckhart,http://upload.wikimedia.org/wikipedia/commons/...
3,Aaron Eckhart,http://25.media.tumblr.com/nJ2vga5sae9o2ks4Flt...
4,Aaron Eckhart,http://upload.wikimedia.org/wikipedia/commons/...
5,Aaron Eckhart,http://media.zenfs.com/en_us/Movies/PhotoG/2nd...
6,Aaron Eckhart,http://img2.timeinc.net/people/i/2008/news/080...
7,Aaron Eckhart,http://latimesblogs.latimes.com/photos/uncateg...
8,Aaron Eckhart,http://collider.com/wp-content/uploads/Aaron-E...
9,Aaron Eckhart,http://movies.dosthana.com/sites/default/files...


# Общая информация о наборе данных

In [43]:
names_actors = unique_labels(actors[1:,0])
names_actresses = unique_labels(actresses[1:,0])
names_all = np.append(names_actors, names_actresses)
names_all = np.array(names_all)

np.savez('base_dir + "/faceScrub/all_names', names_all=names_all)

print("Всего фотографий: ", len(actors) + len(actresses) - 2)
print("Всего фотографий с актерами: ", len(actors) - 1)
print("Всего фотографий с актрисами: ", len(actresses) - 1)
print("Всего актеров:", len(names_actors))
print("Всего актрис:", len(names_actresses))

Всего фотографий:  106863
Всего фотографий с актерами:  55306
Всего фотографий с актрисами:  51557
Всего актеров: 265
Всего актрис: 265


# Загрузка изображений

In [0]:
def url_is_alive(url):
    try:
        urllib.request.urlopen(url)
        return True
    except Exception:
        return False

data_info = []
error_male = []
error_female = []
image_id = 0
pbar = ProgressBar(maxval=len(actors)+len(actresses)-2)
pbar.start()
socket.setdefaulttimeout(5)

# Download actors
for i in range(1,len(actors)+1,1):
  href = actors[i,1]
  if(url_is_alive(href)):
    try:
      urllib.request.urlretrieve(href, folder_for_images+"/{}.jpg".format(image_id))
      people_id = np.where(names_all == actors[i,0])[0][0]
      name = actors[i,0]
      data_info.append([image_id, people_id, name, "male"])
      image_id+=1
    except Exception:
      error_male.append(i)
    pbar.update(i)

# Download actresses
for i in range(1,len(actresses)+1,1):
  href = actresses[i,1]
  if(url_is_alive(href)):
    try:
      urllib.request.urlretrieve(href, folder_for_images+"/{}.jpg".format(image_id))
      people_id = np.where(names_all == actresses[i,0])[0][0]
      name = actresses[i,0]
      data_info.append([image_id, people_id, name, "female"])
      image_id+=1
    except Exception:
      error_female.append(i)
    pbar.update(i)    
      
pbar.finish()

  1% (1602 of 106863) |           | Elapsed Time: 0:14:47 ETA:  1 day, 20:48:49

# Save metadata

In [0]:
data_info = np.array(data_info)
error_male = np.array(error_male)
error_female = np.array(error_female)

np.savez('base_dir + "/faceScrub/download_metadata', data_info=data_info,
         error_male=error_male, error_female=error_female)